In [ ]:
# pip install opencv-python lxml numpy tqdm

In [ ]:
import os
import xml.etree.ElementTree as ET
import glob

# Paths
image_folder = "Wildfire photos/Datacluster Fire and Smoke Sample"
annotation_folder = "Wildfire photos/Annotations"
output_label_folder = "Wildfire photos/YOLO_labels"

# Create output folder if not exists
os.makedirs(output_label_folder, exist_ok=True)

# Class Mapping
# class_mapping = {"fire": 0, "smoke": 1}  # Assign unique IDs to classes - There is no smoke label in the dataset

class_mapping = {"fire": 0}

# Function to convert XML to YOLO format
def convert_xml_to_yolo(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Get image size
    img_width = int(root.find("size/width").text)
    img_height = int(root.find("size/height").text)

    # Get image filename
    filename = root.find("filename").text.replace(".jpg", ".txt")
    output_file = os.path.join(output_label_folder, filename)

    with open(output_file, "w") as f:
        for obj in root.findall("object"):
            class_name = obj.find("name").text
            if class_name not in class_mapping:
                continue  # Skip unknown classes

            class_id = class_mapping[class_name]

            # Get bounding box coordinates
            bndbox = obj.find("bndbox")
            xmin = float(bndbox.find("xmin").text)
            ymin = float(bndbox.find("ymin").text)
            xmax = float(bndbox.find("xmax").text)
            ymax = float(bndbox.find("ymax").text)

            # Convert to YOLO format (normalized)
            x_center = ((xmin + xmax) / 2) / img_width
            y_center = ((ymin + ymax) / 2) / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            # Write to file
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# Convert all XML files
xml_files = glob.glob(os.path.join(annotation_folder, "*.xml"))
for xml_file in xml_files:
    convert_xml_to_yolo(xml_file)

print(f"Saved in '{output_label_folder}'")
